# Initialization

## Packages

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import product

from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression

## Environment

In [3]:
# Environment variables
PROJECTPATH = os.getenv('PROJECTPATH')

## Functions

In [11]:
def prepare_data(clusters, scores, demographics, nk = 2, threshold = 0.5, return_features = False):
    
    nk_col = 'nk{}'.format(nk)
    
    # Compute completion rate for the different assessments
    nparticipants = scores.shape[0]
    completion = dict()
    for col, vals in scores.items():
        if col != 'Subject_ID':
            completion[col] = vals.notna().sum()/nparticipants
            
    # Filter scales above completion threshold
    scales = [col for col, val in completion.items() if val > threshold]        
    
    # Get the scores for the subset of scales
    scores_subset = scores[scales].copy()
    scores_subset['Subject_ID'] = scores['Subject_ID']
    
    # Join scores and cluster information
    clusters_scores = clusters.copy()
    clusters_scores = (clusters_scores
     .rename(columns = {'ID':'file'})
     .loc[:, ['file', nk_col]]
     .merge(demographics, how = 'left', on = 'file'))

    # Filter for POND
    clusters_scores = clusters_scores.loc[clusters_scores['Dataset'] == 'POND']

    # Clean up IDs for merging
    clusters_scores['Subject_ID'] = (clusters_scores['Subject_ID']
                                     .str.replace('sub-', '')
                                     .astype(int))

    # Merge clusters to scores
    clusters_scores = (clusters_scores
                       .loc[:, ['Subject_ID', nk_col]]
                       .merge(scores_subset, on = 'Subject_ID', how = 'left'))

    # Keep only complete observations
    clusters_scores = clusters_scores.dropna()
    
    # Create the input matrix and binary targets
    X = clusters_scores.drop(['Subject_ID', nk_col], axis = 1)
    features = X.columns.to_list()
    X = X.to_numpy()
    y = np.array(clusters_scores[nk_col], dtype = int)-1
    
    if return_features:
        return X,y,features
    else:
        return X,y
    
    
def plot_ROC_curves(X, y, scale = True, outfile = None):
    
    nscales = X.shape[1]
    nparticipants = X.shape[0]
    
    # Maximum number of components
    max_components = X.shape[1]-1
    
    # Iterate over components
    component_range = range(2, max_components)
    list_auc = []
    for nc in component_range:

        # Initiatlize the PLSR module
        plsr = PLSRegression(n_components = nc, scale = scale)

        # Fit the model to the data
        plsr.fit(X, y)

        # Predict cluster labels
        y_pred = plsr.predict(X)

        # Clamp the interval since this isn't a proper classifier
        y_pred[y_pred > 1.0] = 1.0
        y_pred[y_pred < 0] = 0

        # Obtain ROC metrics
        fpr, tpr, roc_thresholds = roc_curve(y, y_pred)

        # Compute AUC
        auc = roc_auc_score(y, y_pred)
        list_auc.append(auc)

        # Plot the ROC curve for a subset of components
        if nc in np.linspace(2, max_components, num = 5, dtype = int):
            plt.plot(fpr, tpr, label = '{} (AUC = {:.2f})'.format(nc, auc))

    # Complete the ROC curve plot for this threshold
    plt.plot(np.linspace(0, 1, 50), np.linspace(0, 1, 50), 
             linestyle = '--', label = 'Chance level (AUC = 0.5)', 
            color = "black")
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('Input features: {}; Participants: {}'.format(nscales, nparticipants))
    plt.legend(title = "Number of components")

    # Output file
    if outfile is not None:
        plt.savefig(outfile, dpi = 300)
    else:
        raise ValueError

    # Close the graphics device
    plt.close()
    
    return list_auc
    

def plot_scores_loadings(X, y, features = None, scale = True, display_loadings = False, 
                         labels = None, palette = None, outfile = None):
    
    if labels is None:
        labels = [None, None]

    # Number of scales and participants
    nscales = X.shape[1]
    nparticipants = X.shape[0]

    # Initiatlize the PLSR module
    plsr = PLSRegression(n_components = 2, scale = scale)

    # Fit the model to the data
    plsr.fit(X, y)

    # Get scores and loadings
    X_pls = plsr._x_scores
    loadings = plsr.x_loadings_

    X_PLS_norm = np.sqrt(np.sum(X_pls**2, axis = 1))
    loadings_norm = np.sqrt(np.sum(loadings**2, axis = 1))

    X_PLS_norm_max = np.max(X_PLS_norm)
    loadings_norm_max = np.max(loadings_norm)
    scale_factor = X_PLS_norm_max/loadings_norm_max

    loadings = loadings*scale_factor

    df_pls = pd.DataFrame(X_pls, columns=['x', 'y'])
    df_pls['cluster'] = [labels[i] for i in y]

    p = sns.jointplot(data = df_pls, x = 'x', y = 'y', hue = 'cluster', 
                      hue_order = labels, palette = palette);
    ax = p.ax_joint
    if display_loadings:
        if features is None:
            raise Exception
        for i in range(nscales):
            ax.arrow(0, 0, loadings[i,0], loadings[i,1])
            ax.text(loadings[i,0], loadings[i,1], features[i], fontsize = 5, horizontalalignment = 'center')

    ax.set_xlabel("Latent variable 1")
    ax.set_ylabel("Latent variable 2")
    p.fig.suptitle('Input features: {}; Participants: {}'.format(nscales, nparticipants))
    p.fig.subplots_adjust(top = 0.95)
    ax.grid()

    # Output file
    if outfile is not None:
        plt.savefig(outfile, dpi = 300)
    else:
        raise ValueError

    # Close the graphics device
    plt.close()


def build_cluster_pairs(k):

    pairs = []
    for pair in product(k, k):
        pair = tuple(sorted(pair))
        if pair[0] != pair[1]:
            if pair not in pairs:
                pairs.append(pair)
    return pairs

***

# Data preparation

Set paths to project directories and import data

In [5]:
# Parameter set ID
params_id = 700

# Output directory
output_dir = os.path.join('outputs', 'human_clinical_multivariate', 'v3', str(params_id))
if not os.path.exists(output_dir): 
    os.makedirs(output_dir)

# Input directories
registration_dir = 'data/human/registration/v3/'
pipeline_dir = 'data/human/derivatives/v3/'

registration_dir = os.path.join(PROJECTPATH, registration_dir)
pipeline_dir = os.path.join(PROJECTPATH, pipeline_dir, str(params_id))

# Demographics file
demographics = os.path.join(registration_dir, 'subject_info', 'demographics.csv')
demographics = pd.read_csv(demographics)

# POND clinical scores
scores = os.path.join(registration_dir, 'subject_info', 'POND', 'POND_clinical_scores_20230915.csv')
scores = pd.read_csv(scores)

# Cluster solutions
cluster_dir = os.path.join(pipeline_dir, 'clusters', 'resolution_3.0')
cluster_file = os.path.join(cluster_dir, 'clusters.csv')
clusters = pd.read_csv(cluster_file)

# Drop columns
cols_to_drop = ['Unnamed: 0', 'site', 'SUB_ID', 
                'DOB', 'PRIMARY_DIAGNOSIS', 
                'RESEARCH_CONFIRM_DIAG', 
                'HSHLD_INCOME_STD', 
                'PRMY_CGVR_STD',
               'SWANPDOC', 'TPOCSPDOC']
scores = scores.drop(cols_to_drop, axis = 1)

# Drop columns containing the following strings
strings_to_drop = ['NSI', 'ETHNCTY', 'EDUC']
for s in strings_to_drop:
    scores = scores.loc[:, ~scores.columns.str.contains(s)]

# Rename the subject ID column for merging
scores = scores.rename(columns = {'subject':'Subject_ID'})

# Assign NaN to missing values 999 code
for col, vals in scores.items():
    x = vals.copy()
    x[x == 999] = np.nan
    scores[col] = x

# Pairwise comparisons across cluster solutions

In [ ]:
# Maximum number of cluster solutions
nk_max = 10

# Cluster solutions
nk_list = list(range(2, nk_max+1))

# Initialize data frame
df_results = pd.DataFrame()

# Iterate over cluster solutions
for nk in nk_list:
    
    # Number of clusters
    klist = list(range(1, nk+1))
    
    # Pairs of clusters
    kpairs = build_cluster_pairs(k = klist)
    
    # Palette for cluster pairs
    palette = {'{}-{}'.format(nk,k):sns.color_palette()[i] for i,k in enumerate(klist)}
    
    # Iterate over cluster pairs
    for k in kpairs:

        # Get cluster IDs
        cluster_ids = ['{}-{}'.format(nk, ki) for ki in k]
        
        # Decrement cluster labels by 1
        k = [x-1 for x in k]

        # Iterate over clinical scale completion thresholds
        thresholds = [0.6, 0.8]
        for threshold in thresholds:

            # Get inputs and labels
            X,y,features = prepare_data(clusters = clusters, 
                               scores = scores, 
                               demographics = demographics,
                               nk = nk,
                               threshold = threshold, 
                                       return_features = True)

            # Filter for participants in the select clusters
            ind_subset = np.isin(y, k)
            X = X[ind_subset,:]
            y = y[ind_subset]

            # Binarize labels
            y[y == k[0]] = 0
            y[y == k[1]] = 1

            # Plot ROC curves at current threshold
            outfile = 'PLSDA_nk{}_{}_{}_ROC_threshold_{}.png'.format(nk, cluster_ids[0], cluster_ids[1], threshold)
            outfile = os.path.join(output_dir, outfile)
            list_auc = plot_ROC_curves(X = X, y = y, outfile = outfile)
            
            # Create a data frame with AUC results
            df_results_i = pd.DataFrame({'auc':list_auc})
            df_results_i['threshold'] = threshold
            df_results_i['cluster1'] = cluster_ids[0]
            df_results_i['cluster2'] = cluster_ids[1]
            df_results_i['nk'] = nk
            df_results_i['cluster1_n'] = len(y) - sum(y)
            df_results_i['cluster2_n'] = sum(y)

            # Concatenate data frames
            df_results = pd.concat([df_results, df_results_i], ignore_index = True)

            # Plot scores and loadings at threshold
            outfile = 'PLSDA_nk{}_{}_{}_scores_threshold_{}.png'.format(nk, cluster_ids[0], cluster_ids[1], threshold)
            outfile = os.path.join(output_dir, outfile)
            plot_scores_loadings(X = X, y = y, 
                                 features = features, display_loadings = True, 
                                 labels = cluster_ids, palette = palette,
                                 outfile = outfile)

In [61]:
# Export results data frame
outfile = 'PLSDA_results.csv'
outfile = os.path.join(output_dir, outfile)
df_results.to_csv(outfile, index=False)